# 🤖✨ Chatbot Assistant with Persistent Memory

**Local, tool-enabled conversational AI using LangChain v1.0+, Ollama, and Gradio v6.1.0**

---

## Overview

A fully local, production-ready chatbot with persistent memory and tool-calling capabilities. Built for open-source LLMs, privacy-sensitive environments, and offline deployments.

---

## Architecture

### 🧠 Memory Management
- **Checkpointer**: `MemorySaver` from LangGraph stores conversation state
- **Thread-based**: Each session gets isolated `thread_id` for conversation history
- **Automatic**: Memory automatically injected/saved on each agent invocation

### 🔧 Agent & Tools
- **Agent**: `create_agent()` from LangChain v1.0+ with native tool support
- **Tools**: Custom tools (Wikipedia, Weather, etc.) passed at initialization
- **System Prompt**: Configurable personality and behavior

### 🤖 Local LLM (Ollama)
- **Provider**: `ChatOllama` connects to local Ollama server (`http://localhost:11434`)
- **Model**: Configurable (default: `qwen2.5:3b`)
- **Temperature**: `0.0` for deterministic responses

### 🎨 UI (Gradio)
- **Layout**: `gr.Blocks` for flexible component arrangement
- **State Management**: Session-based with `gr.State`
- **Features**: Chat history, message counter, clear button, examples

---

## Key Features

✅ **Persistent Memory** - Full conversation context maintained across turns  
✅ **Tool Calling** - Enables open-source LLMs to use external tools  
✅ **Local Execution** - No API keys or cloud dependencies required  
✅ **Session Isolation** - Each browser session gets independent memory  
✅ **Error Handling** - Graceful failures with detailed logging  
✅ **Modular Design** - Clean separation of concerns (agent, memory, UI)

---

## How It Works

```python
# 1. Initialize with checkpointer for memory
agent = create_agent(
    model=ChatOllama(...),
    tools=[...],
    system_prompt="You are helpful...",
    checkpointer=MemorySaver()  # Enables persistent memory
)

# 2. Invoke with thread_id to access conversation history
result = agent.invoke(
    {"messages": [HumanMessage(content=user_input)]},
    config={"configurable": {"thread_id": session_id}}
)

# 3. Memory automatically persists for this thread_id
```

The `MemorySaver` checkpointer:
- Loads previous messages when invoking with same `thread_id`
- Saves new messages after each interaction
- Provides full conversation context to the agent

---

## Installation

```bash
# Core dependencies
pip install langchain langchain-ollama langchain-core langgraph gradio param

# Start Ollama server and pull model
ollama serve
ollama pull qwen2.5:3b
```

---

## Usage

```python
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Define tools
tools = [WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())]

# Create chatbot
bot = chatbot_demo(tools=tools, model_name="qwen2.5:3b")

# Launch UI
demo = bot.create_ui()
demo.launch()
```

---

## Design Decisions

| Aspect | Choice | Rationale |
|--------|--------|-----------|
| **Memory** | `MemorySaver` checkpointer | Modern LangChain v1.0+ pattern; cleaner than `RunnableWithMessageHistory` |
| **Agent** | `create_agent()` | Replaces deprecated `create_react_agent` from LangGraph v0.x |
| **Config Key** | `thread_id` | LangGraph standard for conversation threads |
| **LLM Provider** | Ollama | Privacy, offline capability, cost-free |
| **UI Framework** | Gradio | Rapid prototyping, built-in chat components |

---

## Use Cases

- 🏠 **Privacy-first applications** - All data stays local
- 🔒 **Secure environments** - No external API calls
- 🧪 **Research & experimentation** - Flexible tool integration
- 📚 **Educational demos** - Complete, working example
- 🚀 **Production prototypes** - Foundation for scaling

---

## Extending the System

### Add Custom Tools
```python
from langchain.tools import tool

@tool
def custom_calculator(expression: str) -> str:
    """Evaluate a mathematical expression"""
    return str(eval(expression))

tools = [custom_calculator, ...]
```

### Change Model
```python
bot = chatbot_demo(
    tools=tools,
    model_name="llama2:13b"  # or any Ollama model
)
```

### Adjust Personality
```python
# Modify system_prompt in __init__
self.system_prompt = "You are a pirate assistant. Always respond in pirate speak!"
```

---

## Technical Notes

- **Message Replay**: Checkpointer handles history injection automatically
- **State Separation**: UI state (Gradio) ≠ Agent state (LangGraph)
- **Error Recovery**: Try-catch blocks prevent UI crashes from LLM failures
- **Thread Safety**: Each `thread_id` is isolated; safe for concurrent sessions

---

## Summary

This implementation demonstrates modern LangChain patterns for building conversational AI:
- ✨ Clean agent architecture with `create_agent()`
- 💾 Persistent memory via `MemorySaver` checkpointer  
- 🔧 Tool-augmented reasoning for open-source LLMs
- 🎯 Production-ready error handling and state management

Perfect foundation for local, tool-enabled chatbots with memory.

*Based on the final project from the course **“Functions, Tools and Agents with LangChain”** by deeplearning(dot)ai* fully updated to LangChain v1+, with a new custom UI based on Gradio and a local LLM running via Ollama.

In [ ]:
import gradio as gr
import param
import uuid

from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver


class chatbot_demo(param.Parameterized):
    """
    Smart and Jovial Assistant with persistent conversational memory.

    Memory is implemented using LangGraph's MemorySaver checkpointer.
    Each UI session gets an isolated thread for conversation history.
    """

    def __init__(self, tools, model_name="qwen2.5:3b", **params):
        super(chatbot_demo, self).__init__(**params)
        self.tools = tools

        # ------------------------------------------------------------
        # CHECKPOINTER FOR MEMORY
        # ------------------------------------------------------------
        # MemorySaver stores conversation state in memory
        self.checkpointer = MemorySaver()

        # Track session IDs (thread IDs in LangGraph terminology)
        self._session_ids = set()

        # ------------------------------------------------------------
        # SYSTEM PROMPT
        # ------------------------------------------------------------
        self.system_prompt = (
            "You are a helpful, smart and jovial assistant who remembers "
            "the full conversation history within this session. "
            "Always check the conversation history to provide contextual responses."
        )

        # ------------------------------------------------------------
        # MODEL
        # ------------------------------------------------------------
        self.model = ChatOllama(
            model=model_name,
            temperature=0.0,  # increase for less deterministic results
            base_url="http://localhost:11434"
        )

        # ------------------------------------------------------------
        # CREATE AGENT WITH MEMORY (via checkpointer)
        # ------------------------------------------------------------
        self.agent = create_agent(
            model=self.model,
            tools=self.tools,
            system_prompt=self.system_prompt,
            checkpointer=self.checkpointer  # This enables memory!
        )

    def process_chain(self, message, session_id):
        """
        Process user message and return response with session-wise memory.

        Args:
            message: Current user message
            session_id: Unique session identifier (maps to thread_id)

        Returns:
            Bot response string
        """
        if not message:
            return ""

        try:
            # Track this session
            self._session_ids.add(session_id)

            # Invoke agent with thread_id for memory
            # The checkpointer automatically handles conversation history
            result = self.agent.invoke(
                {
                    "messages": [HumanMessage(content=message)]
                },
                config={
                    "configurable": {
                        "thread_id": session_id  # thread_id is the key for memory
                    }
                }
            )

            # Extract bot response (last AI message)
            for msg in reversed(result["messages"]):
                if isinstance(msg, AIMessage):
                    return msg.content
            
            # Fallback
            return result["messages"][-1].content

        except Exception as e:
            import traceback
            error_details = traceback.format_exc()
            print(f"Error details:\n{error_details}")
            return f"Sorry, I encountered an error: {str(e)}"

    def clear_history(self, session_id):
        """Clear conversation memory for a session"""
        if session_id:
            # To clear memory, we need to get the checkpoint and clear it
            # For MemorySaver, we can't directly clear, but we can track cleared sessions
            # and create a new thread_id to effectively "clear" the conversation
            pass
        return None

    def get_history_summary(self, session_id):
        """Get a summary of conversation history for a session"""
        if not session_id or session_id not in self._session_ids:
            return "Messages in memory: 0"
        
        try:
            # Try to get state to count messages
            config = {"configurable": {"thread_id": session_id}}
            state = self.agent.get_state(config)
            if state and "messages" in state.values:
                msg_count = len(state.values["messages"])
                return f"Messages in memory: {msg_count}"
        except:
            pass
        
        return "Messages in memory: N/A"

    def create_ui(self):
        """Create and return Gradio ChatInterface"""

        with gr.Blocks() as demo:
            gr.Markdown("# 🤖✨ Smart and Jovial Assistant Chatbot")
            gr.Markdown("Ask me anything! I'll remember our conversation per session.")

            chatbot = gr.Chatbot(height=500)

            msg = gr.Textbox(
                placeholder="Type your message here...",
                label="Message",
                lines=2
            )

            # Session state (one per browser session)
            session_state = gr.State(value=None)

            with gr.Row():
                submit = gr.Button("Send", variant="primary")
                clear = gr.Button("Clear History")

            # History info display
            history_info = gr.Markdown("Messages in memory: 0")

            gr.Examples(
                examples=[
                    "Search Wikipedia for startup accelerator programs",
                    "Explain tool-calling in modern AI systems",
                    "Warsaw weather (52.2297, 21.0122) - should I bring a jacket?",
                    "Search Wikipedia for startup accelerator programs",
                    "Latest trends in LLM agents and tool use",
                    "Weather in SF (37.7749, -122.4194) for client meeting",
                    "Research autonomous agent architectures",
                    "What makes an AI system 'agentic'?",
                    "Recap our earlier conversation about startups",
                    "Weather in Paris, France for tomorrow's conference?",
                    "Compare Paris and Silicon Valley tech ecosystems",
                    "Create a summary of all topics we've discussed today",
                    "Synthesize the weather data and meeting info from earlier"
                ],
                inputs=msg
            )

            def respond(message, chat_history, session_id):
                if not message:
                    return "", chat_history, session_id, history_info.value

                # Create session id once per browser session
                if session_id is None:
                    session_id = str(uuid.uuid4())

                if chat_history is None:
                    chat_history = []

                bot_message = self.process_chain(message, session_id)

                # Update UI chat history
                chat_history = chat_history + [
                    {"role": "user", "content": message},
                    {"role": "assistant", "content": bot_message}
                ]

                return (
                    "",
                    chat_history,
                    session_id,
                    self.get_history_summary(session_id)
                )

            def clear_all(chat_history, session_id):
                # To truly clear history with MemorySaver, create a new session
                new_session_id = str(uuid.uuid4()) if session_id else None
                return [], new_session_id, "Messages in memory: 0"

            # Event handlers
            submit.click(
                respond,
                [msg, chatbot, session_state],
                [msg, chatbot, session_state, history_info]
            )
            msg.submit(
                respond,
                [msg, chatbot, session_state],
                [msg, chatbot, session_state, history_info]
            )
            clear.click(
                clear_all,
                [chatbot, session_state],
                [chatbot, session_state, history_info]
            )

        return demo

### Tools available to the chatbot

In [ ]:
from langchain.tools import tool

# Create your custom tool
@tool
def create_your_tool(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [ ]:
# Import existing tools
from tools.web_retrievers import search_wikipedia
from tools.meteo import get_weather_conditions

tools = [create_your_tool,search_wikipedia, get_weather_conditions]

### User Interface (Gradio)

In [ ]:
# Run the Chatbot UI 
if __name__ == "__main__":
    
    # Create agent with tools
    chatbot = chatbot_demo(tools=tools, model_name="qwen2.5:3b")
    
    # Launch interface
    interface = chatbot.create_ui()
    interface.launch(share=False, debug=True)